# Phi-3 Mini + GEPA LoRA Fine-Tuning (Unsloth/PEFT)

This notebook demonstrates an end-to-end Supervised Fine-Tuning (SFT) and optional PPO pipeline for the specified model using GEPA abstention and trace logging. CPU-only smoke tests operate on the tiny dataset subsets under `datasets/`. For full runs, switch to a GPU runtime.

### SetupUncomment the `%pip install` cell if running in a fresh kernel. All dependencies are offline friendly and resolve against the local wheel cache when available.

In [ ]:
# %pip install unsloth transformers accelerate peft trl datasets tqdm pyyaml rich

In [ ]:
import jsonimport mathimport osimport randomfrom pathlib import Pathimport torchfrom datasets import load_datasetfrom peft import LoraConfigfrom transformers import AutoTokenizerimport yamlfrom gepa_mindfulness.core.tracing import SelfTracingLoggerfrom gepa_mindfulness.adapters.self_tracing import format_trace_examplerandom.seed(13)torch.manual_seed(13)

### Load configuration presetsThe YAML preset wires GEPA abstention defaults directly into the training configuration so they remain editable without changing notebook code.

In [ ]:
with open('configs/models/phi3_mini_lora.yaml', 'r', encoding='utf-8') as handle:    model_config = yaml.safe_load(handle)print(json.dumps(model_config, indent=2))

### Dataset preparationWe merge Ethical QA supervision data with OOD stress prompts that label unanswerable scenarios. The formatting function ensures GEPA framing → evidence → decision fields appear in the supervised objective.

In [ ]:
data_dir = Path('datasets')ethical_path = data_dir / 'ethical_qa' / 'sample.jsonl'ood_path = data_dir / 'ood_stress' / 'sample.jsonl'ethical_records = [json.loads(line) for line in ethical_path.read_text().splitlines()]ood_records = [json.loads(line) for line in ood_path.read_text().splitlines()]print(f'Ethical QA records: {len(ethical_records)}')print(f'OOD stress records: {len(ood_records)}')

In [ ]:
def build_sft_example(sample: dict, unanswerable: bool = False) -> dict:    framing = sample['mindfulness_trace'].get('framing', '')    evidence = sample['mindfulness_trace'].get('evidence', '')    decision = sample['mindfulness_trace'].get('decision', '')    safeguards = sample['mindfulness_trace'].get('safeguards', '')    prompt = (        '### GEPA Framing\n' + framing + '\n\n'        '### GEPA Evidence\n' + evidence + '\n\n'        '### GEPA Decision\n' + decision + '\n\n'        '### GEPA Safeguards\n' + safeguards + '\n\n'        '### User Question\n' + sample['question']    )    target = sample['answer']    return {        'prompt': prompt,        'response': target,        'abstain': unanswerable,    }

In [ ]:
train_examples = [build_sft_example(sample) for sample in ethical_records]for record in ood_records:    mapped = {        'question': record['scenario'],        'answer': record['expected_behavior'],        'mindfulness_trace': {            'framing': 'Clarify scenario and risks',            'evidence': 'Refer to safety policies and uncertainty handling',            'decision': 'Choose honest, careful response',            'safeguards': 'Escalate or abstain when unsure',        },    }    train_examples.append(build_sft_example(mapped, unanswerable=record.get('unanswerable', False)))print(train_examples[0]['prompt'])print('Total SFT examples:', len(train_examples))

### Tokeniser and model initialisationUse Unsloth to prepare the base model with quantisation aware loading. The LoRA config mirrors the YAML preset and writes adapters to `outputs/`.

In [ ]:
model_name = model_config['model_name']print('Loading model:', model_name)print('Phi-3 Mini defaults to 4-bit loading for memory efficiency.')tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)if tokenizer.pad_token is None:    tokenizer.pad_token = tokenizer.eos_tokenlora_cfg = LoraConfig(**model_config['lora'])print(lora_cfg)

### Confidence head and GEPA tracing hooksDuring generation we attach a lightweight confidence estimator that mirrors the abstention rule: confidence < 0.75 triggers the fallback text `I don't know.` Token metadata is stored in `runs/tokens.jsonl` and checkpoints in `runs/trace.jsonl`.

In [ ]:
runs_dir = Path('runs')tokens_path = runs_dir / 'tokens.jsonl'trace_path = runs_dir / 'trace.jsonl'summary_path = runs_dir / 'summary.json'tracer = SelfTracingLogger(base_path=runs_dir)ABSTENTION_THRESHOLD = model_config['gepa']['abstention_threshold']ABSTENTION_TEXT = model_config['gepa']['abstention_text']def compute_confidence(logits: torch.Tensor) -> float:    probs = torch.softmax(logits, dim=-1)    top_conf = float(probs.max().item())    return top_conf

### Supervised fine-tuning loopFor CPU smoke tests we run only a handful of optimisation steps. Replace the placeholders with the real Unsloth trainer when running on GPU. Each batch logs GEPA events via the Self-Tracing adapter.

In [ ]:
max_steps = min(3, model_config['train']['max_steps'])loss_history = []for step, example in enumerate(train_examples[:max_steps]):    with tracer.trace(chain='sft') as trace:        tracer.log_event('framing', example['prompt'], module='PromptBuilder')        tracer.log_event('decision', example['response'], module='ReferenceAnswer')    loss_history.append(0.0)print('Completed smoke loop with steps:', len(loss_history))summary_path.write_text(json.dumps({'loss_history': loss_history}, indent=2))

### Evaluation and abstention metricsWe simulate evaluation logits and enforce the abstention threshold. This cell ensures the notebook demonstrates at least one abstention event.

In [ ]:
abstentions = 0for example in train_examples[:4]:    logits = torch.randn(1, tokenizer.vocab_size)    conf = compute_confidence(logits[0])    if conf < ABSTENTION_THRESHOLD:        print(ABSTENTION_TEXT)        abstentions += 1    else:        print('Model answer:', example['response'][:60])print('Total abstentions:', abstentions)

### Persist trace artefactsToken and checkpoint files enable the offline viewer. This toy example emits deterministic fake token records so downstream tooling has consistent schemas.

In [ ]:
runs_dir.mkdir(exist_ok=True)tokens_path.write_text('\n'.join(json.dumps({'token': 'demo', 'conf': 0.9}) for _ in range(5)))trace_path.write_text('\n'.join(json.dumps({'stage': 'decision', 'content': 'demo'}) for _ in range(3)))print('Wrote artefacts to', runs_dir.resolve())

### Optional: PPO fine-tuning with TRLSet `ENABLE_PPO = True` to execute a short rollout loop. The reward combines the task objective with GEPA scores and honesty signals.

In [ ]:
ENABLE_PPO = Falseif ENABLE_PPO:    from trl import PPOTrainer, AutoModelForCausalLMWithValueHead    import yaml    with open('configs/ppo/ppo_default.yaml', 'r', encoding='utf-8') as handle:        ppo_cfg = yaml.safe_load(handle)    print('Loaded PPO config:', ppo_cfg)    # Placeholder setup for PPO trainer using the same tokenizer and model.    # Insert rollout + reward computation here when running on GPU.    for _ in range(2):        tracer.log_event('reflection', 'PPO step placeholder', module='PPO')    print('Completed PPO placeholder loop.')else:    print('Skipping PPO smoke test; enable when resources allow.')

### Quickstart: score and render reportsThe final cell mirrors the CLI instructions from the README and produces both the HTML score summary and the interactive viewer.

In [ ]:
!gepa score --trace runs/trace.jsonl --policy policies/default_cw4.yml --out report.html!gepa view --trace runs/trace.jsonl --tokens runs/tokens.jsonl --out report_view.html